In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("audit_10%.csv")

In [3]:
df

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,NaN,0.500,6.68,NaN,0.2,...,0.4,NaN,0.2,0.0,2.4,8.574,0.4,0.5,1.7148,1.0
1,3.89,0.00,NaN,0.000,4.83,NaN,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.4,0.0,NaN,0.0,NaN,1.416,NaN,NaN,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,NaN,NaN,0.2,NaN,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,NaN,2.0,1.578,0.4,0.5,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,NaN,0.074,NaN,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,NaN,0.0
773,55.57,0.24,0.2,NaN,NaN,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,NaN,0.040,0.00,0.2,0.000,0.20,5.0,NaN,...,0.4,NaN,0.2,0.0,2.0,NaN,0.4,0.5,0.2880,0.0


In [4]:
df.dtypes

Sector_score      float64
PARA_A            float64
Score_A           float64
Risk_A            float64
PARA_B            float64
Score_B           float64
Risk_B            float64
TOTAL             float64
numbers           float64
Score_B.1         float64
Risk_C            float64
Money_Value       float64
Score_MV          float64
Risk_D            float64
District_Loss     float64
PROB              float64
RiSk_E            float64
History           float64
Prob              float64
Risk_F            float64
Score             float64
Inherent_Risk     float64
CONTROL_RISK      float64
Detection_Risk    float64
Audit_Risk        float64
Risk              float64
dtype: object

In [5]:
df.isnull().sum()

Sector_score      74
PARA_A            62
Score_A           76
Risk_A            87
PARA_B            77
Score_B           83
Risk_B            93
TOTAL             94
numbers           88
Score_B.1         94
Risk_C            91
Money_Value       71
Score_MV          84
Risk_D            71
District_Loss     80
PROB              63
RiSk_E            93
History           73
Prob              89
Risk_F            80
Score             75
Inherent_Risk     71
CONTROL_RISK      92
Detection_Risk    75
Audit_Risk        81
Risk               0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 20176
nilai hilang : 2017
persentase : 9.997026169706581


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Sector_score       9.536082
PARA_A             7.989691
Score_A            9.793814
Risk_A            11.211340
PARA_B             9.922680
Score_B           10.695876
Risk_B            11.984536
TOTAL             12.113402
numbers           11.340206
Score_B.1         12.113402
Risk_C            11.726804
Money_Value        9.149485
Score_MV          10.824742
Risk_D             9.149485
District_Loss     10.309278
PROB               8.118557
RiSk_E            11.984536
History            9.407216
Prob              11.469072
Risk_F            10.309278
Score              9.664948
Inherent_Risk      9.149485
CONTROL_RISK      11.855670
Detection_Risk     9.664948
Audit_Risk        10.438144
Risk               0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,5)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
0,3.89,4.18,0.6,2.508,2.50,0.0,0.500,6.68,0.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.4,8.574,0.4,0.5,1.7148
1,3.89,0.00,0.0,0.000,4.83,0.0,0.966,4.83,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.0,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.2,0.4,0.0,0.0,0.0,0.0,1.416,0.0,0.0,0.2832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,0.00,0.00,0.2,0.000,0.40,0.2,0.080,0.89,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.578,0.4,0.5,0.3156
772,55.57,0.47,0.2,0.094,0.37,0.0,0.074,0.00,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.0000
773,55.57,0.24,0.2,0.000,0.00,0.2,0.008,0.28,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912
774,55.57,0.20,0.0,0.040,0.00,0.2,0.000,0.20,5.0,0.0,...,0.2,0.4,0.0,0.2,0.0,2.0,0.000,0.4,0.5,0.2880


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 4780


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,5)

Kolom : 1
[2.72, 3.89, 3.89, 3.89, 3.41]
mean :  3.56
standar deviasi :  0.4593038210161112
fuzzy :  [(2.72, 0.18783684251892757), (3.89, 0.772534168179087), (3.89, 0.772534168179087), (3.89, 0.772534168179087), (3.41, 0.9480745734793711)]
fuzzy :  None
pembilang :  12.759324249866085
penyebut :  3.45351392053556
rata-rata tertimbang :  3.6945918109655134
 
[55.57, 55.57, 59.85, 3.89, 2.72]
mean :  35.519999999999996
standar deviasi :  26.352422279555253
fuzzy :  [(55.57, 0.7487056169651749), (55.57, 0.7487056169651749), (59.85, 0.6530147838946866), (3.89, 0.48663138444875365), (2.72, 0.460925842550275)]
fuzzy :  None
pembilang :  125.44079146284894
penyebut :  3.097983244824065
rata-rata tertimbang :  40.49111358895446
 
[2.37, 2.37, 55.57, 1.99, 55.57]
mean :  23.574
standar deviasi :  26.124993090908177
fuzzy :  [(2.37, 0.7193963536059231), (2.37, 0.7193963536059231), (55.57, 0.47241328062830473), (1.99, 0.7108790382994693), (55.57, 0.47241328062830473)]
fuzzy :  None
pembilang :  5

[3.41, 3.41, 3.41, 3.41, 3.41]
mean :  3.41
standar deviasi :  0.0
median :  3
fuzzy :  [(3.41, 3.41), (3.41, 3.41), (3.41, 3.41), (3.41, 3.41), (3.41, 3.41)]
fuzzy :  3.41
pembilang :  58.14050000000001
penyebut :  17.05
rata-rata tertimbang :  3.41
 
[1.99, 1.99, 1.85, 1.99, 1.85]
mean :  1.934
standar deviasi :  0.06858571279792894
fuzzy :  [(1.99, 0.7165560753439455), (1.99, 0.7165560753439455), (1.85, 0.47240328690554645), (1.99, 0.7165560753439455), (1.85, 0.47240328690554645)]
fuzzy :  None
pembilang :  6.025731931353876
penyebut :  3.094474799842929
rata-rata tertimbang :  1.9472551308738184
 
[55.57, 55.57, 55.57, 3.89, 55.57]
mean :  45.233999999999995
standar deviasi :  20.671999999999997
fuzzy :  [(55.57, 0.8825083402897203), (55.57, 0.8825083402897203), (55.57, 0.8825083402897203), (3.89, 0.13536335042701722), (55.57, 0.8825083402897203)]
fuzzy :  None
pembilang :  196.6905173127601
penyebut :  3.665396711585898
rata-rata tertimbang :  53.66145407700181
 
[0.0, 0.0, 1.99, 

[0.0, 1.85, 1.85, 1.85, 1.85]
mean :  1.48
standar deviasi :  0.7400000000000001
fuzzy :  [(0.0, 0.13536335042701736), (1.85, 0.8825083402897205), (1.85, 0.8825083402897205), (1.85, 0.8825083402897205), (1.85, 0.8825083402897205)]
fuzzy :  None
pembilang :  6.5305617181439315
penyebut :  3.665396711585899
rata-rata tertimbang :  1.7816793738864811
 
[0.0, 1.99, 2.72, 55.57, 55.57]
mean :  23.169999999999998
standar deviasi :  26.469468449517457
fuzzy :  [(0.0, 0.6817600876050527), (1.99, 0.7260760138280943), (2.72, 0.7419930897619543), (55.57, 0.4728043056752542), (55.57, 0.4728043056752542)]
fuzzy :  None
pembilang :  56.01058300441818
penyebut :  3.095437802545609
rata-rata tertimbang :  18.094559340961883
 
[0.0, 1.85, 1.85, 1.85, 1.85]
mean :  1.48
standar deviasi :  0.7400000000000001
fuzzy :  [(0.0, 0.13536335042701736), (1.85, 0.8825083402897205), (1.85, 0.8825083402897205), (1.85, 0.8825083402897205), (1.85, 0.8825083402897205)]
fuzzy :  None
pembilang :  6.5305617181439315
pen

[0.0, 9.332515631187238, 21.61, 55.57, 1.99]
mean :  17.700503126237447
standar deviasi :  20.393792650689797
fuzzy :  [(0.0, 0.6861786656620541), (9.332515631187238, 0.9192725266950567), (21.61, 0.981795143414201), (55.57, 0.17837200014703994), (1.99, 0.7432712170795048)]
fuzzy :  None
pembilang :  41.18696004404271
penyebut :  3.5088895529978563
rata-rata tertimbang :  11.737890127905054
 
[0.0, 55.57, 1.99, 55.57, 55.57]
mean :  33.739999999999995
standar deviasi :  26.743585399119542
fuzzy :  [(0.0, 0.45124332087116437), (55.57, 0.71668832387627), (1.99, 0.49428177206152313), (55.57, 0.71668832387627), (55.57, 0.71668832387627)]
fuzzy :  None
pembilang :  120.46273119981541
penyebut :  3.095590064561498
rata-rata tertimbang :  38.91430347282737
 
[0.0, 55.57, 17.96171901165554, 3.89, 0.0]
mean :  15.484343802331107
standar deviasi :  21.104292619795242
fuzzy :  [(0.0, 0.7640406696331751), (55.57, 0.1646904219035754), (17.96171901165554, 0.9931345052115962), (3.89, 0.859937726978592

[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0,

...

[0.0, 0.5, 0.5, 0.5, 0.5]
mean :  0.4
standar deviasi :  0.20000000000000004
fuzzy :  [(0.0, 0.13536335042701728), (0.5, 0.8825083402897206), (0.5, 0.8825083402897206), (0.5, 0.8825083402897206), (0.5, 0.8825083402897206)]
fuzzy :  None
pembilang :  1.7650166805794412
penyebut :  3.6653967115858994
rata-rata tertimbang :  0.4815349659152652
 
[0.0, 0.5, 0.5, 0.0, 0.5]
mean :  0.3
standar deviasi :  0.24494897427831783
fuzzy :  [(0.0, 0.4724032869055458), (0.5, 0.7165560753439463), (0.5, 0.7165560753439463), (0.0, 0.4724032869055458), (0.5, 0.7165560753439463)]
fuzzy :  None
pembilang :  1.0748341130159194
penyebut :  3.0944747998429305
rata-rata tertimbang :  0.34733975312078025
 
[0.0, 0.5, 0.5, 0.5, 0.0]
mean :  0.3
standar deviasi :  0.24494897427831783
fuzzy :  [(0.0, 0.4724032869055458), (0.5, 0.7165560753439463), (0.5, 0.7165560753439463), (0.5, 0.7165560753439463), (0.0, 0.4724032869055458)]
fuzzy :  None
pembilang :  1.0748341130159194
penyebut :  3.0944747998429305
rata-rata t

[1.6104, 0.92, 0.3208, 0.3156, 0.3148]
mean :  0.69632
standar deviasi :  0.5132437019584362
fuzzy :  [(1.6104, 0.20478560924946188), (0.92, 0.9094110366668305), (0.3208, 0.7651865746608807), (0.3156, 0.7594968875864226), (0.3148, 0.7586184029963939)]
fuzzy :  None
pembilang :  1.8904270430055679
penyebut :  3.3974985111599896
rata-rata tertimbang :  0.5564173278651797
 
[9.1172, 17.3976, 8.3872, 21.4224, 9.8836]
mean :  13.2416
standar deviasi :  5.2163179228264065
fuzzy :  [(9.1172, 0.7315796176841336), (17.3976, 0.7280708605559819), (8.3872, 0.6485733667947223), (21.4224, 0.2923891827052149), (9.8836, 0.8128692782215136)]
fuzzy :  None
pembilang :  39.07411066175358
penyebut :  3.2134823059615663
rata-rata tertimbang :  12.159429223949463
 
[13.3564, 57.3328, 13.8436, 0.0, 23.0832]
mean :  21.5232
standar deviasi :  19.358109767226757
fuzzy :  [(13.3564, 0.9148618996976292), (57.3328, 0.18072203034283343), (13.8436, 0.9243336675801372), (0.0, 0.5390027766120506), (23.0832, 0.9967585

[0.0, 0.28, 0.28, 0.294, 0.28]
mean :  0.22680000000000003
standar deviasi :  0.11352955562319443
fuzzy :  [(0.0, 0.13598223039071955), (0.28, 0.896029624477067), (0.28, 0.896029624477067), (0.294, 0.8393190941322657), (0.28, 0.896029624477067)]
fuzzy :  None
pembilang :  0.9994246982356224
penyebut :  3.663390197954186
rata-rata tertimbang :  0.27281415416620086
 
[0.0, 0.4104, 0.4104, 0.0, 0.3992]
mean :  0.244
standar deviasi :  0.19926713728058623
fuzzy :  [(0.0, 0.4725525323744286), (0.4104, 0.7056567247060072), (0.4104, 0.7056567247060072), (0.0, 0.4725525323744286), (0.3992, 0.7383957869958946)]
fuzzy :  None
pembilang :  0.8739706378074519
penyebut :  3.0948143011567666
rata-rata tertimbang :  0.28239840997270266
 
[0.0, 0.2904, 0.0, 0.298, 0.2884]
mean :  0.17536000000000002
standar deviasi :  0.1432166694208464
fuzzy :  [(0.0, 0.47258055365003787), (0.2904, 0.724278987080248), (0.0, 0.47258055365003787), (0.298, 0.6930811749922233), (0.2884, 0.7323769830497124)]
fuzzy :  None

[0.0, 0.288, 0.2824, 0.0, 0.19505905125171957]
mean :  0.15309181025034393
standar deviasi :  0.1292720245425006
fuzzy :  [(0.0, 0.49600747169944315), (0.288, 0.5801356918566313), (0.2824, 0.606392429846755), (0.0, 0.49600747169944315), (0.19505905125171957, 0.9486730762787765)]
fuzzy :  None
pembilang :  0.5233715716504217
penyebut :  3.1272161413810493
rata-rata tertimbang :  0.16736021687944122
 
[0.0, 0.28, 0.28, 0.2824, 0.19505905125171957]
mean :  0.20749181025034394
standar deviasi :  0.10893441776628482
fuzzy :  [(0.0, 0.163028487040625), (0.28, 0.8013179107866888), (0.28, 0.8013179107866888), (0.2824, 0.7894623018412613), (0.19505905125171957, 0.9935089280308944)]
fuzzy :  None
pembilang :  0.8654750929923372
penyebut :  3.548635538486158
rata-rata tertimbang :  0.24388954109430674
 
[0.0, 14.1292, 57.3328, 13.8436, 16.07180134695847]
mean :  20.275480269391693
standar deviasi :  19.396684289865814
fuzzy :  [(0.0, 0.5791022618907872), (14.1292, 0.9510401890377126), (57.3328, 0

[0.0, 0.3156, 0.2892, 0.3224, 0.2996]
mean :  0.24535999999999997
standar deviasi :  0.12323495607983961
fuzzy :  [(0.0, 0.13781788909921844), (0.3156, 0.8500883406740277), (0.2892, 0.9386899385116666), (0.3224, 0.8225184930294499), (0.2996, 0.9076927813188086)]
fuzzy :  None
pembilang :  1.0768817299701068
penyebut :  3.656807442633171
rata-rata tertimbang :  0.29448685687280063
 
[0.0, 0.4104, 0.3992, 0.3944, 0.3912]
mean :  0.31904
standar deviasi :  0.1596527431646572
fuzzy :  [(0.0, 0.13581405634955143), (0.4104, 0.8489852562027638), (0.3992, 0.8815848980689079), (0.3944, 0.8945880061820896), (0.3912, 0.9029096345151092)]
fuzzy :  None
pembilang :  1.406395999115249
penyebut :  3.663881851318422
rata-rata tertimbang :  0.3838540805045794
 
[0.0, 0.2964, 0.2968, 0.2988, 0.292836]
mean :  0.23696719999999996
standar deviasi :  0.11849919283843245
fuzzy :  [(0.0, 0.13543460480394043), (0.2964, 0.8818254276013207), (0.2968, 0.8803289052111793), (0.2988, 0.8727351418900022), (0.292836,

In [15]:
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
0,3.890000,4.180000,0.600000,2.508000,2.50,0.214772,0.500,6.68,5.0,0.200000,...,0.2,0.4,0.0,0.2,0.0,2.4,8.574000,0.4,0.500000,1.71480
1,3.890000,0.287058,0.214772,0.102932,4.83,0.194305,0.966,4.83,5.0,0.200000,...,0.2,0.4,0.0,0.2,0.0,2.0,2.554000,0.4,0.500000,0.51080
2,3.890000,0.510000,0.200000,0.102000,0.23,0.200000,0.046,0.74,5.0,0.200000,...,0.2,0.4,0.0,0.2,0.0,2.0,1.548000,0.4,0.500000,0.30960
3,3.890000,3.144170,0.200000,1.043569,10.80,0.600000,6.480,10.80,6.0,0.600000,...,0.2,0.4,0.0,0.2,0.0,4.4,17.530000,0.4,0.500000,3.50600
4,3.890000,0.430098,0.200000,0.088372,0.08,0.200000,0.016,0.08,5.0,0.200000,...,0.2,0.4,0.0,0.2,0.0,2.0,1.416000,0.4,0.481535,0.28320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,18.572889,0.000000,0.200000,0.000000,0.40,0.200000,0.080,0.89,5.0,0.200000,...,0.2,0.4,0.0,0.2,0.0,2.0,1.578000,0.4,0.500000,0.31560
772,55.570000,0.470000,0.200000,0.094000,0.37,0.192614,0.074,0.00,5.0,0.200000,...,0.2,0.4,0.0,0.2,0.0,2.0,1.568000,0.4,0.500000,0.29774
773,55.570000,0.240000,0.200000,0.000000,0.00,0.200000,0.008,0.28,5.0,0.200000,...,0.2,0.4,0.0,0.2,0.0,2.0,1.456000,0.4,0.500000,0.29120
774,55.570000,0.200000,0.190754,0.040000,0.00,0.200000,0.000,0.20,5.0,0.192614,...,0.2,0.4,0.0,0.2,0.0,2.0,1.393037,0.4,0.500000,0.28800


In [16]:
dummy.describe()

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
count,776.000000,776.00000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,...,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,20.142626,2.27999,0.348120,1.262066,9.227663,0.308415,6.261187,12.871708,5.017324,0.221126,...,0.204099,0.506067,0.096649,0.213991,0.045361,2.674324,15.781770,0.558848,0.495539,6.442744
std,23.873896,5.64156,0.173990,3.430685,22.191725,0.169441,30.067481,51.016067,0.350229,0.078148,...,0.034320,0.275338,0.522712,0.066148,0.287016,0.867549,40.154958,0.424462,0.024049,37.062784
min,1.285157,0.00000,0.061064,0.000000,0.000000,0.061064,0.000000,0.000000,2.391785,0.118323,...,0.061064,0.122128,0.000000,0.061064,0.000000,1.006359,0.427449,0.236646,0.152660,0.011699
25%,2.370000,0.00000,0.200000,0.000000,0.000000,0.200000,0.000000,0.557500,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,2.000000,1.575500,0.400000,0.500000,0.315100
50%,3.890000,0.76000,0.200000,0.147000,0.350000,0.200000,0.056000,1.345000,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,2.200000,2.200000,0.400000,0.500000,0.531741
75%,55.570000,2.22000,0.600000,1.311000,3.882500,0.400000,1.348879,7.412500,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,3.200000,10.591000,0.400000,0.500000,3.295700
max,59.850000,85.00000,0.600000,51.000000,144.390000,0.600000,758.778000,1268.910000,9.000000,0.600000,...,0.600000,2.400000,9.000000,0.600000,5.400000,5.200000,583.688000,5.800000,0.500000,961.514400


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [34]:
x = dummy.values
y = df.iloc[:, -1].values

In [50]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1001)

In [51]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [52]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[121   0]
 [  1  72]]
0.9948453608247423
